https://hesperia.gsfc.nasa.gov/rhessi3/data-access/rhessi-data/flare-list/index.html

http://sprg.ssl.berkeley.edu/~jimm/hessi/hsi_flare_list.html

In [ ]:
from dataclasses import dataclass
import numpy as np
import scipp as sc
import pandas as pd
from datetime import date, datetime, time
import re

In [ ]:
infile = 'hessi_flare_list.txt'

In [ ]:
def parse_month(m) -> int:
    return ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'].index(m) + 1

def parse_date(d) -> date:
    day, month, year = d.split('-')
    return date(day=int(day), month=parse_month(month), year=int(year))

def parse_time(t) -> time:
    return time.fromisoformat(t)

def parse_datetime(d, t) -> np.datetime64:
    dt = datetime.combine(parse_date(d), parse_time(t))
    return np.datetime64(int(dt.timestamp()), 's')

In [ ]:
def get_quality(flags: list) -> int:
    pattern = re.compile(r'Q(\d)')
    for flag in flags:
        if match := pattern.match(flag):
            return int(match[1])
    return -1

In [ ]:
@dataclass
class Entry:
    flare_id: int
    peak_time: np.datetime64
    duration: float
    total_counts: float
    energy_range: str
    x: float
    y: float
    radial: float
    eclipsed: bool
    non_solar: bool
    quality: int
    
    @classmethod
    def parse(cls, s) -> Entry:
        fields = [c for c in s.strip().split(' ') if c]
        flags=fields[13:]
        eclipsed = 'ED' in flags or 'EE' in flags or 'ES' in flags
        non_solar = 'NS' in flags
        quality = get_quality(flags)

        return cls(
        flare_id=int(fields[0]),
        peak_time=parse_datetime(fields[1], fields[3]),
        duration=float(fields[5]),
        total_counts=float(fields[7]),
        energy_range=fields[8],
        x=float(fields[9]),
        y=float(fields[10]),
        radial=float(fields[11]),
        eclipsed=eclipsed,
        non_solar=non_solar,
        quality=quality)

In [ ]:
def load_txt(fname):
    flare_id = []
    peak_time = []
    duration = []
    total_counts = []
    energy_range = []
    x_pos = []
    y_pos = []
    radial = []
    eclipsed = []
    origin = []
    quality = []
    
    # Use to remove duplicates.
    # Way faster than searching through flare_id for every line.
    seen = set()

    with open(fname, 'r') as f:
        for _ in range(7):
            f.readline()

        while line := f.readline().strip():
            entry = Entry.parse(line)
            if entry.quality == -1:
                continue
            if entry.flare_id in seen:
                continue
            seen.add(entry.flare_id)
            
            flare_id.append(entry.flare_id)
            peak_time.append(entry.peak_time)
            duration.append(entry.duration)
            total_counts.append(entry.total_counts)
            energy_range.append(entry.energy_range)
            x_pos.append(entry.x)
            y_pos.append(entry.y)
            radial.append(entry.radial)
            eclipsed.append(entry.eclipsed)
            origin.append('non_solar' if entry.non_solar else 'solar')
            quality.append(entry.quality)
            
    return sc.DataArray(
        sc.array(dims=['event'], values=total_counts, unit='count'),
        coords={
            'time': sc.array(dims=['event'], values=peak_time, unit='s'),
            'duration': sc.array(dims=['event'], values=duration, unit='s'),
            'x': sc.array(dims=['event'], values=x_pos, unit='asec'),
            'y': sc.array(dims=['event'], values=y_pos, unit='asec'),
            'radial': sc.array(dims=['event'], values=radial, unit='asec'),
        },
        attrs={
            'energy_range': sc.array(dims=['event'], values=energy_range),
            'eclipsed': sc.array(dims=['event'], values=eclipsed),
            'origin': sc.array(dims=['event'], values=origin),
            'quality':sc.array(dims=['event'],values=quality),
        }
    )

da = load_txt(infile)

In [ ]:
da.masks['pos'] = (da.coords['x'] == 0*sc.Unit('asec')) & (da.coords['y'] == 0*sc.Unit('asec'))

In [ ]:
da

In [ ]:
sc.bin(da, edges=[sc.linspace('x', -700, 700, 100, unit='asec')]).plot()

In [ ]:
sc.bin(da, edges=[sc.linspace('y', -700, 700, 100, unit='asec')]).plot()

In [ ]:
sc.bin(da, edges=[sc.linspace('radial', 0, 700, 100, unit='asec')]).plot()